# Numerical Analysis Project
Team: 아프지마

Member

* 20181628 성창훈
* 20181635 신명철

written by 성창훈, 신명철

# Google Colab

 AutoKeras는 GPU 가속을 지원하므로, 원활한 학습을 위해서 Google Colab의 GPU를 이용해서 학습한다.

Google Colab에서는 Tesla K80과 Tesla T4 두 종류의 GPU를 제공한다.

이 노트에서는 K80보다 성능이 더 좋은 Tesla T4를 이용한다.

In [10]:
!nvidia-smi

Mon May 24 20:07:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    29W /  70W |    478MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Packages

대다수의 Web-Based notebook 서비스들(Google Colab, Kaggle, Paperspace...)은 docker를 이용해 일관된 환경을 제공한다.

환경의 업데이트는 패키지의 업데이트보다 보수적으로 이루어지므로 보통 버전이 낮다.

이때 일부 최신 패키지는 높은 버전의 pip 버전을 요하므로, 반드시 패키지 관리 도구를 최신 버전으로 업데이트하여 사용한다.

업데이트 후에는 kernel을 다시 시작한다. (초기화가 아님에 유의한다)

In [11]:
!pip install autokeras

# AutoKeras

익숙한 도구인 Keras의 AutoML 버전이다.

Keras가 가진 강력한 도구들을 양껏 활용했다는 사실을 쉽게 눈치챌 수 있었다.

AutoKeras는 ImageClassifier를 제공하여 이미지에 전문화된 모델을 자동으로 탐색하고(NAS), 하이퍼파라메터를 최적화한다(HPO).

In [12]:
import os
import shutil

import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

import autokeras as ak

# Data Loading
Kaggle의 Leaf Rice Disease Dataset을 이용한다.

Keras의 ImageDagaGenerator의 메소드와 유사하게 생긴 image_dataset_from_directory() 메소드를 통해 디렉토리로부터 손쉽게 학습 데이터와 훈련데이터를 불러올 수 있다.

이미지와 배치의 사이즈 정도만 지정해주면 된다.

구글 드라이브에 3:1 비율로 구서한 학습 데이터와 검증 데이터를 압축해 업로드했고, 이를 노트북에서 불러와 이용했다.

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
!unzip /content/drive/MyDrive/archive_60_20.zip -d /content/split_data

Archive:  /content/drive/MyDrive/archive_60_20.zip
replace /content/split_data/test/blast/IMG_0682.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [18]:
train_data_dir = os.path.join('/content/split_data', "train")
test_data_dir = os.path.join('/content/split_data', "test")
print(train_data_dir)
print(test_data_dir)

/content/split_data/train
/content/split_data/test


In [19]:
batch_size = 8
img_height = 480
img_width = 360

train_data = ak.image_dataset_from_directory(
    train_data_dir,
    # Use 20% data as testing data.
    validation_split=0.2,
    subset="training",
    image_size=(img_width, img_height),
    batch_size=batch_size,
)

test_data = ak.image_dataset_from_directory(
    test_data_dir,
    validation_split=0.2,
    subset="validation",
    image_size=(img_width, img_height),
    batch_size=batch_size,
)

Found 180 files belonging to 3 classes.
Using 144 files for training.
Found 60 files belonging to 3 classes.
Using 12 files for validation.


## Build a classifier and Fit
AutoKeras가 Imgage classification에 적합한 모델을 탐색한다.

코드가 간단한 것이 AutoML의 큰 장점인데, 모델 설계와 학습을 단 두 줄에 끝냈다.

시간과 자원을 고려해 최대 탐색 시간은 30분으로 제한하려 했지만, time_limit 매개변수가 deprecated되고 max_trials 매개변수를 이용해달라는 커밋을 발견할 수 있었다.

몇 번의 시도 끝에 우리가 의도한 소요시간을 갖는 trials와 epoch값을 찾아냈다.

기본적으로 fit() 메소드에서 verbose를 지원하므로 실시간으로 얼마나 어떻게 학습했는지 확인할 수 있다.


In [20]:
# Initialize the image classifier.
clf = ak.ImageClassifier(overwrite=True,max_trials=3)
# Feed the image classifier with training data.
clf.fit(train_data,epochs = 20)

Trial 3 Complete [00h 17m 23s]
val_loss: 0.13765773177146912

Best val_loss So Far: 0.13765773177146912
Total elapsed time: 00h 29m 35s
INFO:tensorflow:Oracle triggered exit
Epoch 1/20
Not enough memory, reduce batch size to 4.
Epoch 1/20
36/36 [==============================] - 49s 1s/step - loss: 1.0049 - accuracy: 0.4931
Epoch 2/20
36/36 [==============================] - 49s 1s/step - loss: 0.6664 - accuracy: 0.8611
Epoch 3/20
36/36 [==============================] - 49s 1s/step - loss: 0.4352 - accuracy: 0.9236
Epoch 4/20
36/36 [==============================] - 49s 1s/step - loss: 0.3029 - accuracy: 0.9306
Epoch 5/20
36/36 [==============================] - 49s 1s/step - loss: 0.1968 - accuracy: 0.9861
Epoch 6/20
36/36 [==============================] - 49s 1s/step - loss: 0.1472 - accuracy: 0.9792
Epoch 7/20
36/36 [==============================] - 49s 1s/step - loss: 0.1095 - accuracy: 0.9931
Epoch 8/20
36/36 [==============================] - 48s 1s/step - loss: 0.0773 - accur

# Get accuracy of the final model

가장 좋은 성능을 가진 모델로 테스트를 진행한다.

가장 오래걸리고 가장 고생했던 패키지여서 그런지, 가장 높은 성능을 보여주었다.

test accuracy는 100%로, 처음 보는 데이터를 완벽하게 분류해냈다.

무수히 많은 경고메세지가 나타났지만, 검색해본 결과 실행에는 문제가 없었다.



In [21]:
# Evaluate the best model with testing data.
print(clf.evaluate(test_data))

2/2 [==============================] - 5s 425ms/step - loss: 0.0683 - accuracy: 1.0000
[0.06825975328683853, 1.0]
